In [111]:
import os
import chromadb
from tqdm import tqdm
from langchain_community.chat_models import ChatOllama
from langchain_community.llms import Ollama
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [ ]:
'''
from flask import Flask, request
from gradio_client import Client

app = Flask(__name__)
app.config['CORS_HEADERS'] = 'Content-Type'


@app.route('/cs_llm/', methods=['GET'])

prompt = request.args.get('prompt')
'''

In [9]:
llm = ChatOllama(model="llama2:7b")
emb = OllamaEmbeddings(model="llama2:7b")

In [29]:
llm.invoke("Hello world!")

"Hello there! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [30]:
class OllamaEmbeddingFn(chromadb.EmbeddingFunction):
    def __call__(self, input: chromadb.Documents) -> chromadb.Embeddings:
        return emb.embed_documents(input)

In [31]:
# initialise chroma collection
# adapted from https://docs.trychroma.com/usage-guide#using-collections
client = chromadb.PersistentClient(path="documents/chroma_db")
collection = client.get_or_create_collection(
    name="helpsheets", embedding_function=OllamaEmbeddingFn()
)

In [65]:
# paths of documents to index
helpsheet_dir = r"documents"
helpsheet_paths = []
for root, dirs, files in os.walk(helpsheet_dir):
    for file in files:
        helpsheet_paths.append(os.path.join(root, file))
print(helpsheet_paths[26])

documents\LN\05.SortingA.pdf


In [66]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
print(collection.count())
# index documents (takes forever)
# adapted from https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf/#using-pypdf
if collection.count() == 4832:
    for i, hs_path in tqdm(enumerate(helpsheet_paths)):
        print(i, hs_path)
        if 'pdf' not in hs_path:
            continue
        if i<=26:
            continue #to save time in debugging

        # load document
        loader = PyPDFLoader(hs_path)
        docs = loader.load()
        # split document into chunks
        splits = text_splitter.split_documents(docs)
        # unique ids for each chunk
        ids = [f"{i} - {j}" for j in range(len(splits))]
        # add chunks into chroma collection
        collection.add(
            ids=ids,
            metadatas=[d.metadata for d in splits],
            documents=[d.page_content for d in splits],
        )

4832


0it [00:00, ?it/s]Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 49 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 62 0 (offset 0)
Ignoring wrong pointing object 64 0 (offset 0)
Ignoring wrong pointing object 66 0 (offset 0)
Ignoring wrong pointing object 73 0 (offset 0)
Ignoring wrong pointing object 75 0 (offset 0)
Ignoring wrong pointing object 77 0 (offset 0)
Ignoring wrong pointing object 88 0 (offset 0)
Ignoring wrong pointing object 93 0 (offset 0)
Ignoring wrong pointing object 98 0 (offset 0)
Ignoring wrong pointing object 114 0 (offset 0)
Ignoring wrong pointing object 116 0 (offse

0 documents\Introduction.to.Algorithms.4th.Leiserson.Stein.Rivest.Cormen.MIT.Press.9780262046305.EBooksWorld.ir.pdf
1 documents\chroma_db\chroma.sqlite3
2 documents\chroma_db\217e7dbe-4914-4272-a617-d692bbc469c3\data_level0.bin
3 documents\chroma_db\217e7dbe-4914-4272-a617-d692bbc469c3\header.bin
4 documents\chroma_db\217e7dbe-4914-4272-a617-d692bbc469c3\index_metadata.pickle
5 documents\chroma_db\217e7dbe-4914-4272-a617-d692bbc469c3\length.bin
6 documents\chroma_db\217e7dbe-4914-4272-a617-d692bbc469c3\link_lists.bin
7 documents\helpsheet collection\1 - Complexities and Searching.pdf
8 documents\helpsheet collection\10 - Graphs.pdf
9 documents\helpsheet collection\11 - Graph Operations and Analysis.pdf
10 documents\helpsheet collection\12 - Minimum Spanning Tree.pdf
11 documents\helpsheet collection\13 - Single Source Shortest Path.pdf
12 documents\helpsheet collection\2 - Sorting.pdf
13 documents\helpsheet collection\3 - Arrays and Linked Lists.pdf
14 documents\helpsheet collection\4 

28it [08:20, 17.87s/it]Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 46 0 (offset 0)
Ignoring wrong pointing object 48 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 59 0 (offset 0)
Ignoring wrong pointing object 61 0 (offset 0)
Ignoring wrong pointing object 63 0 (offset 0)
Ignoring wrong pointing object 65 0 (offset 0)
Ignoring wrong pointing object 131 0 (offset 0)
Ignoring wrong pointing object 133 0 (offset 0)
Ignoring wrong pointing object 135 0 (offset 0)
Ignoring wrong pointing object 137 0 (offset 0)
Ignoring wrong pointing object 139 0 (offset 0)
Ignoring wrong pointing object 141 0 (offset 0)
Ignoring wrong pointing object 143 0 (offset 0)
Ignoring wrong pointing object 1

28 documents\LN\07.SortingC.QuickSort.pdf


...

64it [2:03:40, 38.54s/it]

64 documents\questions_wo_answer\ps6.pdf


65it [2:04:52, 48.72s/it]

65 documents\questions_wo_answer\ps8.pdf


66it [2:05:29, 114.08s/it]


In [112]:
# initialise langchain vector store from chroma client
# adapted from https://python.langchain.com/docs/integrations/vectorstores/chroma/#passing-a-chroma-client-into-langchain
vectorstore = Chroma(
    client=client,
    collection_name="helpsheets",
    embedding_function=emb,
)
retriever = vectorstore.as_retriever()
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

In [166]:
#Ignore the cell
helper_prompt = "Identify the possible key concepts in the following question. DO NOT answer it. Only return the highly relevant specific concepts in a list without title."
contextualize_helper_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", helper_prompt),
        ("human", "{input}"),
    ]
)
keyword_identifying_chain = create_stuff_documents_chain(llm, contextualize_helper_prompt)

ValueError: Prompt must accept context as an input variable. Received prompt with input variables: ['chat_history', 'input']

In [228]:
# code for chat history (including next 2 code blocks)
# adapted from https://python.langchain.com/docs/use_cases/question_answering/chat_history/
contextualize_q_system_prompt = "Given a chat history and the latest user question \
which might reference context in the chat history, \
pinpoint the specific key concepts mentioned in the latest user question. DO NOT answer it. Only return the highly relevant concepts as a list without title. \
Exclude general terms like 'data structure' or 'algorithms,' focusing instead on precise keywords such as 'QuickSort,' 'Heap,' or 'Binary Tree.' \
Do NOT retrieve irrelevant contexts or chat history."
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, compression_retriever, contextualize_q_prompt
)

In [229]:
qa_system_prompt = """You are an computer science tutor for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If it's a question about definition, use five sentences maximum and keep the answer concise.\
If it's a problem-solving questions, DIVIDE your answer into clearly labelled 'hint' and 'solution' sections, answer in the format "Hint: (your hints). Possible Solution: (Your Solution)".\
DO NOT answer any questions in the context.\
You may use your own knowledge as well, but you need to state clearly which part is from your own knowledge. \
Do not mention "the context". \
If you don't know the answer or the question is NOT RELEVANT to Data Structure or Algorithms, just say this single sentence"Hmm, this problem seems to be out of syllabus. Please further check with your tutor." Don't try to make up an answer. \

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [230]:
### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [231]:
# streams llm output
# adapted from https://python.langchain.com/docs/use_cases/question_answering/streaming/
def query_and_print(input: str, session_id="abc123"):
    for chunk in conversational_rag_chain.stream(
        {"input": input},
        config={
            "configurable": {"session_id": session_id}
        },  # constructs a key "abc123" in `store`.
    ):
        if "answer" in chunk:
            print(chunk["answer"], end="")
        else:
            print(chunk)

In [232]:
query_and_print("Could you provide a definition of HashMap?")

{'input': 'Could you provide a definition of HashMap?'}
{'chat_history': []}


c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'context': [Document(page_content='The relevant parts of the context are:\n\n* "Every 32-bit integer gets a unique hash code."\n* "Note: hashcode is always a 32-bit integer."\n\nThese parts of the context are relevant to the question because they provide information about the type of hash code used in the program and its size, which is important for understanding how hash tables work.', metadata={'page': 72, 'source': 'documents\\LN\\15.Hashing2.pdf'}), Document(page_content='NO_OUTPUT. The question is asking for a definition of HashMap, and the context provided does not contain any information related to HashMap or its definition.', metadata={'page': 89, 'source': 'documents\\LN\\01.Introduction.pdf'}), Document(page_content='The relevant part of the context for answering the question "Could you provide a definition of HashMap?" is:\n\n* "What is a graph?"', metadata={'page': 60, 'source': 'documents\\LN\\17.Hashing4GraphsIntro.pdf'}), Document(page_content='The following part of the

In [233]:
query_and_print("Could you solve this problem? Given an integer array nums sorted in non-decreasing order, remove some duplicates in-place such that each unique element appears at most twice. The relative order of the elements should be kept the same. Since it is impossible to change the length of the array in some languages, you must instead have the result be placed in the first part of the array nums. More formally, if there are k elements after removing the duplicates, then the first k elements of nums should hold the final result. It does not matter what you leave beyond the first k elements. Return k after placing the final result in the first k slots of nums.Do not allocate extra space for another array. You must do this by modifying the input array in-place with O(1) extra memory")

{'input': 'Could you solve this problem? Given an integer array nums sorted in non-decreasing order, remove some duplicates in-place such that each unique element appears at most twice. The relative order of the elements should be kept the same. Since it is impossible to change the length of the array in some languages, you must instead have the result be placed in the first part of the array nums. More formally, if there are k elements after removing the duplicates, then the first k elements of nums should hold the final result. It does not matter what you leave beyond the first k elements. Return k after placing the final result in the first k slots of nums.Do not allocate extra space for another array. You must do this by modifying the input array in-place with O(1) extra memory'}
{'chat_history': [HumanMessage(content='Could you provide a definition of HashMap?'), AIMessage(content='Of course! A HashMap is a data structure that stores key-value pairs in an array, where each key is 

c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'context': [Document(page_content='The relevant parts of the context for answering the question are:\n\n* "in-place modification"\n* "O(1) extra memory"\n\nThese two phrases are mentioned in the question and suggest that the algorithm being discussed is designed to modify a data structure in place, without requiring additional memory beyond what is already stored in the structure.', metadata={'page': 1309, 'source': 'documents\\Introduction.to.Algorithms.4th.Leiserson.Stein.Rivest.Cormen.MIT.Press.9780262046305.EBooksWorld.ir.pdf'}), Document(page_content='NO OUTPUT. The given context does not contain any relevant information to answer the question.', metadata={'page': 1302, 'source': 'documents\\Introduction.to.Algorithms.4th.Leiserson.Stein.Rivest.Cormen.MIT.Press.9780262046305.EBooksWorld.ir.pdf'}), Document(page_content='The following parts of the context are relevant to answering the question:\n\n* The use of a `HashMap` to store points.\n* The use of a `Radix sort` to sort the p

In [234]:
query_and_print("What is HashMap?")

{'input': 'What is HashMap?'}
{'chat_history': [HumanMessage(content='Could you provide a definition of HashMap?'), AIMessage(content='Of course! A HashMap is a data structure that stores key-value pairs in an array, where each key is associated with a specific index in the array. The hash code of a key is used to determine which index the corresponding value should be stored at. The size of the array is fixed and is typically chosen to be a power of 2 for performance reasons. This allows for efficient searching of values based on their keys, as the hash code can quickly locate the appropriate index in the array.'), HumanMessage(content='Could you solve this problem? Given an integer array nums sorted in non-decreasing order, remove some duplicates in-place such that each unique element appears at most twice. The relative order of the elements should be kept the same. Since it is impossible to change the length of the array in some languages, you must instead have the result be placed 

c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'context': [Document(page_content='The relevant parts of the context are:\n\n* `HashMap`: This is mentioned in the question as a data structure that can be used for in-place modification.\n* `Non-decreasing order`: This is also mentioned in the question as a requirement for the algorithm to satisfy.\n* `Tarjan’s ofüine lowest-common-ancestors algorithm`: This is mentioned in the context as a task parallelism algorithm that can be used to solve the problem.', metadata={'page': 1309, 'source': 'documents\\Introduction.to.Algorithms.4th.Leiserson.Stein.Rivest.Cormen.MIT.Press.9780262046305.EBooksWorld.ir.pdf'}), Document(page_content='The following parts of the context are relevant to the question:\n\n* "Data structure" - Relevant because a HashMap is a type of data structure.\n* "Hash code" - Relevant because a hash code is used in the context of a HashMap.\n* "Non-decreasing order" - Relevant because a HashMap is a non-decreasing ordered data structure.\n* "In-place modification" - Rel

In [235]:
query_and_print("What is MCT?")

{'input': 'What is MCT?'}
{'chat_history': [HumanMessage(content='Could you provide a definition of HashMap?'), AIMessage(content='Of course! A HashMap is a data structure that stores key-value pairs in an array, where each key is associated with a specific index in the array. The hash code of a key is used to determine which index the corresponding value should be stored at. The size of the array is fixed and is typically chosen to be a power of 2 for performance reasons. This allows for efficient searching of values based on their keys, as the hash code can quickly locate the appropriate index in the array.'), HumanMessage(content='Could you solve this problem? Given an integer array nums sorted in non-decreasing order, remove some duplicates in-place such that each unique element appears at most twice. The relative order of the elements should be kept the same. Since it is impossible to change the length of the array in some languages, you must instead have the result be placed in t

c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'context': [Document(page_content='The relevant parts of the context that are relevant to answer the question are:\n\n* The three implementation options for inserting an element into a data structure: (1) Sorted array-insert, which takes O(n) time for search and O(log n) time for insertion; (2) Unsorted array-insert, which takes O(1) time for search and O(n) time for insertion; and (3) Linked list-insert, which takes O(1) time for search and O(n) time for insertion.\n\nTherefore, the answer to the question is:\n\nOption 2: Unsorted array-insert.', metadata={'page': 60, 'source': 'documents\\LN\\08.Trees.pdf'}), Document(page_content='The relevant parts of the context that are relevant to answer the question are:\n\n* Data structure: HashMap\n* In-place modification\n* Array\n\nThese parts are relevant because they relate to the specific question being asked, which is about modifying a data structure in place.', metadata={'page': 1, 'source': 'documents\\LN\\07.SortingC.QuickSort.pdf'}

In [236]:
query_and_print("Given an integer array nums, return all the triplets [nums[i], nums[j], nums[k]] such that i != j, i != k, and j != k, and nums[i] + nums[j] + nums[k] == 0. Notice that the solution set must not contain duplicate triplets.")

{'input': 'Given an integer array nums, return all the triplets [nums[i], nums[j], nums[k]] such that i != j, i != k, and j != k, and nums[i] + nums[j] + nums[k] == 0. Notice that the solution set must not contain duplicate triplets.'}
{'chat_history': [HumanMessage(content='Could you provide a definition of HashMap?'), AIMessage(content='Of course! A HashMap is a data structure that stores key-value pairs in an array, where each key is associated with a specific index in the array. The hash code of a key is used to determine which index the corresponding value should be stored at. The size of the array is fixed and is typically chosen to be a power of 2 for performance reasons. This allows for efficient searching of values based on their keys, as the hash code can quickly locate the appropriate index in the array.'), HumanMessage(content='Could you solve this problem? Given an integer array nums sorted in non-decreasing order, remove some duplicates in-place such that each unique elem

c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\lsq69.PC-20230731LVFT\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'context': [Document(page_content='The following parts of the context are relevant to answer the question:\n\n* The problem statements mentioning the algorithms to be tested (SorterA, SorterB, SorterC, and SorterD) and their properties (unique elements, duplicates, array size, time complexity, space complexity).\n* The request for evidence to prove the identity of each sorter.', metadata={'page': 2, 'source': 'documents\\questions_wo_answer\\ps3.pdf'}), Document(page_content='The relevant parts of the context that are relevant to answer the question are:\n\n* Resize takes too long to find items to copy.\n* Inserting, deleting, and searching are more expensive in a big table.', metadata={'page': 35, 'source': 'documents\\LN\\16.Hashing3.pdf'}), Document(page_content='The relevant parts of the context that are relevant to answering the question are:\n\n* The data structure being used is an array.\n* The size of the array is 1531.\n* The array contains unique elements.\n* There are dupli

In [27]:
store.clear()